<a href="https://colab.research.google.com/github/LEESUAJE1978/deeplearning_sejong/blob/master/07_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. 텐서 플로우 버전 선택

In [ ]:
#0. 텐서플로우 버전 선택
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 1.x
except Exception:
    pass


#https://sshkim.tistory.com/153
#http://complx.me/2016-12-31-practical-seq2seq/


TensorFlow 1.x selected.


1. 패키지 불러오기

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent = 4) #데이터를 조금 더 보기 좋게 출력
sess = tf.InteractiveSession()

In [ ]:
pprint.PrettyPrinter()

2. 문자 데이터 생성: 원핫인코딩(one hot encoding)

In [ ]:
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

3. Cell 생성: input.shape(1,1,4), output(1,1,2) hidden size = output size

In [ ]:
sess = tf.InteractiveSession() #세션 열기

with tf.variable_scope('one_cell') as scope:
  #One Cell RNN input_dim(4 -> output_dim(2)
  hidden_size = 2
  cell = tf.nn.rnn_cell.BasicRNNCell(num_units= hidden_size, activation= 'tanh')
  #cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
  print(cell.output_size, cell.state_size)
  
  x_data = np.array([[h]], dtype=np.float32)
  #print(x_data)
  #pp.pprint(x_data)
  outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())

2 2
array([[[-0.39074057,  0.54641414]]], dtype=float32)


/tensorflow-1.15.2/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
tf.reset_default_graph() #초기화 

4. 시퀀스 생성(Sequence Generation) 5 sequence case, unfolding

In [ ]:
with tf.variable_scope('five_squences') as scope:
  #one cell RNN input_dim(4) -> output_dim(2). sequence:5
  hidden_size = 2
  cell = tf.nn.rnn_cell.BasicRNNCell(num_units= hidden_size, activation= 'tanh')
  #cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
  x_data = np.array([[h,e,l,l,o]], dtype=np.float32)
  print(x_data.shape)
  pp.pprint(x_data)
  #print(x_data)
  outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  print("output:", outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
output: [[[-0.37510583  0.45925424]
  [-0.5939723   0.82202166]
  [-0.2795796   0.8758886 ]
  [-0.40267962  0.81773204]
  [-0.19215012  0.81724626]]]


5. 3 batch 추가하기

In [ ]:
with tf.variable_scope('3_batches') as scope:
  x_data = np.array([[h,e,l,l,o],
                     [e,o,l,l,l],
                     [l,l,e,e,l]], dtype = np.float32)
  pp.pprint(x_data)
  hidden_size = 2
  cell = tf.nn.rnn_cell.BasicRNNCell(num_units= hidden_size, activation= 'tanh', reuse=True)
  outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.3928772 ,  0.28243938],
        [ 0.47264907, -0.52239746],
        [ 0.15453476,  0.68916184],
        [ 0.7547219 , -0.14099404],
        [ 0.5561704 ,  0.5437676 ]],

       [[ 0.37296227, -0.12942494],
        [ 0.5007543 ,  0.32722932],
        [ 0.65996814,  0.31488758],
        [ 0.67467415,  0.41632968],
        [ 0.7152587 ,  0.37042257]],

       [[ 0.4116896 ,  0.18737596],
        [ 0.5843126 ,  0.33954635],
        [ 0.649735  ,  0.05574571],
        [ 0.5218184 ,  0.2738814 ],
        [ 0.6398249 ,  0.357773  ]]], dtype=float32)


6. batch 별 Sequence Variation

In [ ]:
with tf.variable_scope('3_batches') as scope:
  x_data = np.array([[h,e,l,l,o],
                     [e,o,l,l,l],
                     [l,l,e,e,l]], dtype = np.float32)
  pp.pprint(x_data)
  hidden_size = 2
  #cell = tf.nn.rnn_cell.BasicRNNCell(num_units= hidden_size, activation= 'tanh')
  cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
  outputs, _states = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,4,2], dtype=tf.float32)
  sess.run(tf.global_variables_initializer())
  pp.pprint(outputs.eval())
  

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
array([[[ 0.58715814,  0.33818203],
        [ 0.11921459,  0.01006974],
        [-0.38844317,  0.51570195],
        [-0.48579237, -0.13499208],
        [ 0.862683  , -0.78295153]],

       [[ 0.6258553 , -0.2680938 ],
        [ 0.6479483 , -0.06772559],
        [-0.5724471 ,  0.78078264],
        [-0.565644  , -0.41279504],
        [ 0.        ,  0.        ]],

       [[-0.32123303,  0.44532415],
        [-0.47132123, -0.04031159],
        [ 0.        ,  0.        ],
        [ 0.        ,  

7. flatten based softmax

In [ ]:
x_data = np.arange(45, dtype=np.float32).reshape(3,5,3)
pp.pprint(x_data) 

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)


In [ ]:
hidden_size = 3
sequence_length = 5
batch_size = 3
num_classes = 5

x_data = np.arange(45, dtype=np.float32).reshape(-1, hidden_size) #15 X 3
print(x_data.shape) 
softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
print(softmax_w.shape)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes)
pp.pprint(outputs)
print(outputs.shape)

(15, 3)
(3, 5)
array([[[  25.,   28.,   31.,   34.,   37.],
        [  70.,   82.,   94.,  106.,  118.],
        [ 115.,  136.,  157.,  178.,  199.],
        [ 160.,  190.,  220.,  250.,  280.],
        [ 205.,  244.,  283.,  322.,  361.]],

       [[ 250.,  298.,  346.,  394.,  442.],
        [ 295.,  352.,  409.,  466.,  523.],
        [ 340.,  406.,  472.,  538.,  604.],
        [ 385.,  460.,  535.,  610.,  685.],
        [ 430.,  514.,  598.,  682.,  766.]],

       [[ 475.,  568.,  661.,  754.,  847.],
        [ 520.,  622.,  724.,  826.,  928.],
        [ 565.,  676.,  787.,  898., 1009.],
        [ 610.,  730.,  850.,  970., 1090.],
        [ 655.,  784.,  913., 1042., 1171.]]], dtype=float32)
(3, 5, 5)


8. Loss 계산하기

In [ ]:
y_data = tf.constant([[1,1,1]])
sess = tf.Session()
print(sess.run(y_data))

[[1 1 1]]


In [ ]:
prediction = tf.constant([[[0.2,0.7], [0.6,0.2],[0.2,0.9]]], dtype = tf.float32)
weights = tf.constant([[1,1,1]], dtype = tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Loss:  0.5967595
